[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/worldbank/OpenNightLights/blob/master/onl/tutorials/mod6_4_intro_to_GHSL.ipynb)

# Intro to Global Human Settlement Layer

Identifying areas of human settlement is a large area of focus in Earth Observation and many other disciplines.

The Global Human Settlement Layer (GHSL) dataset is a useful resource for understanding areas of settlement and happily for our purposes, it is available in the Google Earth Engine (GEE) catalogue.

The dataset we are particularly interested in is the GHSL "Settlement Grid" layer. The settlement grids in this dataset have been generated via the GHSL built-up areas and population grids, which are themselves derived from Landsat image collections and other sources and these layers are also available on GEE.

More details and links to resources are available on the <a href="https://developers.google.com/earth-engine/datasets/catalog/JRC_GHSL_P2016_SMOD_POP_GLOBE_V1#description">collection's landing pages</a>, including a description and link to the methodology.

A couple things to note. 

First, there is one band with four "degrees of urbanization":
- 0. Inhabited areas (
- 1. Rural grid cells
- 2. Low Density Clusters (towns and cities)
- 3. High Density Clusters (cities)
    
We made a choice in framing our analysis that we are interested in the change of the Low and High Density clusters ("built up") relative to everything else, so we will classify any pixel with values in [3,4] as "built up" and assign this `1` or not and assign it `0`.

A second item worth noting is that the spatial resolution for this grid layer is 1000 meters.

Let's take a quick look at the data...

In [1]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [2]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# get our Nepal boundary
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME','Nepal')).geometry()

ghsl = ee.ImageCollection('JRC/GHSL/P2016/SMOD_POP_GLOBE_V1').filter(ee.Filter.date('2015-01-01', '2015-12-31')).select('smod_code').median();

# create a boolean mask setting anything equaling 2 (low density) or 3 (high density) as True
# this will actually be our binary label layer
ghslbinary = ghsl.gte(2)

ghslVis= {"min":0.0, "max":3.0,"palette":['000000', '448564', '70daa4', 'ffffff']}
ghslbiVis= {"palette":['000000', 'ffffff']}

map1 = geemap.Map()
map1.centerObject(aoi,7)
map1.addLayer(ghsl.clip(aoi), ghslVis, 'Degree of Urbanization')
map1.addLayer(ghslbinary.clip(aoi), ghslbiVis, 'Built up')
map1

Map(center=[28.268224502649357, 83.97637443188768], controls=(WidgetControl(options=['position', 'transparent_…

In the original data layer ("Degree of Urbanization") you can see the high density patch of Kathmandu (white) as well as the low density cluster classifications that follow what appears to be major road networks and other towns (bright green). We can also see the rural areas (green) and the rest of the country (black).

In our binary mask ("Built up"), you can see how the coverage is over both low and high cluster areas. This is the layer we will use as our labels for the training data.